In [2]:
import cv2
import mediapipe as mp
import pygame
import sys
import math
import random
import time

# 손 회전 방향을 위한 각도 변수
Right_degree = 9.5
Left_degree = 13

# 색깔들
colors = [
    (0, 0, 0),
    (120, 37, 179),
    (100, 179, 179),
    (80, 34, 22),
    (80, 134, 22),
    (180, 34, 22),
    (180, 34, 122),
]


class Figure:
    x = 0
    y = 0

    figures = [
        [[1, 5, 9, 13], [4, 5, 6, 7]],
        [[4, 5, 9, 10], [2, 6, 5, 9]],
        [[6, 7, 9, 10], [1, 5, 6, 10]],
        [[1, 2, 5, 9], [0, 4, 5, 6], [1, 5, 9, 8], [4, 5, 6, 10]],
        [[1, 2, 6, 10], [5, 6, 7, 9], [2, 6, 10, 11], [3, 5, 6, 7]],
        [[1, 4, 5, 6], [1, 4, 5, 9], [4, 5, 6, 9], [1, 5, 6, 9]],
        [[1, 2, 5, 6]],
    ]

    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.type = random.randint(0, len(self.figures) - 1)
        self.color = random.randint(1, len(colors) - 1)
        self.rotation = 0

    def image(self):
        return self.figures[self.type][self.rotation]

    def rotate(self, direction): # 방향에 따라 회전하게 변형
        if direction == 'right':
            self.rotation = (self.rotation + 1) % len(self.figures[self.type])
        if direction == 'left':
            self.rotation = (self.rotation - 1) % len(self.figures[self.type])

    def get_rightmost_x(self):
        max_x = 0
        for i in range(4):
            for j in range(4):
                if i * 4 + j in self.image():
                    max_x = max(max_x, j)
        return max_x + self.x

    def get_leftmost_x(self):
        min_x = 3  # 3으로 초기화해서 더 작은 값을 찾도록 설정
        for i in range(4):
            for j in range(4):
                if i * 4 + j in self.image():
                    min_x = min(min_x, j)
        return min_x + self.x


class Tetris:
    def __init__(self, height, width):
        self.level = 2
        self.score = 0
        self.state = "start"
        self.field = []
        self.height = 0
        self.width = 0
        self.x = 100
        self.y = 60
        self.zoom = 30
        self.figure = None
    
        self.height = height
        self.width = width
        self.field = []
        self.score = 0
        self.state = "start"
        for i in range(height):
            new_line = []
            for j in range(width):
                new_line.append(0)
            self.field.append(new_line)

    def new_figure(self):
        self.figure = Figure(3, 0)

    def intersects(self):
        intersection = False
        for i in range(4):
            for j in range(4):
                if i * 4 + j in self.figure.image():
                    if i + self.figure.y > self.height - 1 or \
                            j + self.figure.x > self.width - 1 or \
                            j + self.figure.x < 0 or \
                            self.field[i + self.figure.y][j + self.figure.x] > 0:
                        intersection = True
        return intersection

    def break_lines(self):
        lines = 0
        for i in range(1, self.height):
            zeros = 0
            for j in range(self.width):
                if self.field[i][j] == 0:
                    zeros += 1
            if zeros == 0:
                lines += 1
                for i1 in range(i, 1, -1):
                    for j in range(self.width):
                        self.field[i1][j] = self.field[i1 - 1][j]
        self.score += lines ** 2

    def go_down(self):
        self.figure.y += 1
        if self.intersects():
            self.figure.y -= 1
            self.freeze()

    def freeze(self):
        for i in range(4):
            for j in range(4):
                if i * 4 + j in self.figure.image():
                    self.field[i + self.figure.y][j + self.figure.x] = self.figure.color
        self.break_lines()
        self.new_figure()
        if self.intersects():
            self.state = "gameover"
            

    def rotate(self, direction):
        old_rotation = self.figure.rotation
        self.figure.rotate(direction)
        if self.intersects():
            self.figure.rotation = old_rotation

    

# 주먹인지 확인하는 함수
def is_rock(landmarks):
    index_tip = landmarks[mp_hands.HandLandmark.INDEX_FINGER_TIP].y
    middle_tip = landmarks[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y
    ring_tip = landmarks[mp_hands.HandLandmark.RING_FINGER_TIP].y
    pinky_tip = landmarks[mp_hands.HandLandmark.PINKY_TIP].y

    index_mcp = landmarks[mp_hands.HandLandmark.INDEX_FINGER_MCP].y
    middle_mcp = landmarks[mp_hands.HandLandmark.MIDDLE_FINGER_MCP].y
    ring_mcp = landmarks[mp_hands.HandLandmark.RING_FINGER_MCP].y
    pinky_mcp = landmarks[mp_hands.HandLandmark.PINKY_MCP].y

    return (index_tip > index_mcp or
            middle_tip > middle_mcp or
            ring_tip > ring_mcp or
            pinky_tip > pinky_mcp)

# 손 회전을 확인하는 함수
def get_hand_rotation(landmarks):
    # 손목과 손가락 사이의 벡터를 계산하여 손의 회전 방향을 추정합니다.

    # 손목 랜드마크의 좌표를 가져옵니다.
    wrist = landmarks[mp_hands.HandLandmark.WRIST]
    
    # 검지 손가락과 중지 손가락의 MCP 랜드마크의 좌표를 가져옵니다.
    index_mcp = landmarks[mp_hands.HandLandmark.INDEX_FINGER_MCP]
    middle_mcp = landmarks[mp_hands.HandLandmark.MIDDLE_FINGER_MCP]

    # 손목에서 검지 손가락 MCP까지의 벡터를 계산합니다.
    wrist_to_index = (index_mcp.x - wrist.x, index_mcp.y - wrist.y)
    
    # 손목에서 중지 손가락 MCP까지의 벡터를 계산합니다.
    wrist_to_middle = (middle_mcp.x - wrist.x, middle_mcp.y - wrist.y)

    # 두 벡터의 방향을 비교하여 각도를 계산합니다.
    angle_index = math.atan2(wrist_to_index[1], wrist_to_index[0])
    angle_middle = math.atan2(wrist_to_middle[1], wrist_to_middle[0])

    # 각도 차이 계산
    angle_diff = angle_index - angle_middle
    
    # 각도를 -180도에서 180도 범위로 조정합니다.
    angle_diff_degrees = math.degrees(angle_diff)

    # 각도를 기준으로 손의 회전 방향을 결정합니다.
    # angle_diff가 양수이면 오른쪽으로 회전한 것으로 간주합니다.
    # angle_diff가 음수이면 왼쪽으로 회전한 것으로 간주합니다.
    if angle_diff_degrees > Left_degree:  # 임계값을 설정하여 오른쪽 회전으로 간주할 범위를 지정
        return 'left'
    elif angle_diff_degrees < Right_degree:  # 임계값을 설정하여 왼쪽 회전으로 간주할 범위를 지정
        return 'right'
    else:
        return 'neutral'


# Initialize the game engine
pygame.init()

# Define some colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
GRAY = (128, 128, 128)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)

size = (500, 700)
screen = pygame.display.set_mode(size)
pygame.display.set_caption("Hand Landmarks with Pygame")

# Loop until the user clicks the close button.
done = False
clock = pygame.time.Clock()
fps = 25
game = Tetris(20, 10)
counter = 0
STOP = False
color = GREEN

# Mediapipe 손 모듈 초기화
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# 손 위치 관련 변수
num_landmarks = 21
hand_x = [0] * num_landmarks
hand_y = [0] * num_landmarks

overcount = 0

# 웹캠을 통해 비디오 캡처 시작
webcam = cv2.VideoCapture(0)
if not webcam.isOpened():
    print("Could not open webcam")
    sys.exit()

with mp_hands.Hands(
    model_complexity=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
    max_num_hands=1
) as hands:
    while not done:
        screen.fill(WHITE)

        counter += 1
    
        if game.figure is None:
            game.new_figure()

        if counter > 100000:
            counter = 0
    
        if counter % (fps // game.level // 2) == 0 and not STOP:
            if game.state == "start":
                game.go_down()
    
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                done = True
                webcam.release()
                pygame.quit()
                sys.exit()
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    game.__init__(20, 10)
                    overcount = 0

        # 웹캠에서 프레임 읽기
        status, frame = webcam.read()
        if not status:
            print("웹캠에서 프레임을 가져올 수 없습니다.")
            break

        # 성능 향상을 위해 이미지를 RGB로 변환
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Mediapipe로 손 랜드마크 추출
        results = hands.process(frame_rgb)

        # 랜드마크가 감지되면 Pygame 화면에 그리기
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # 주먹을 쥐고 있을 때 블록의 x 위치를 설정하는 코드:
                if is_rock(hand_landmarks.landmark):                
                    figure_x_position = int(((size[0] - hand_x[9]) - game.x) / game.zoom) - 1
                
                    # 오른쪽 경계를 넘지 않도록 설정 (블록의 가장 오른쪽 칸을 기준으로)
                    if figure_x_position + game.figure.get_rightmost_x() - game.figure.x >= game.width:
                        figure_x_position = game.width - (game.figure.get_rightmost_x() - game.figure.x) - 1
                
                    # 왼쪽 경계를 넘지 않도록 설정 (블록의 가장 왼쪽 칸을 기준으로)
                    if figure_x_position + game.figure.get_leftmost_x() - game.figure.x < 0:
                        figure_x_position = -game.figure.get_leftmost_x()
                
                    game.figure.x = figure_x_position
                    
                    direction = get_hand_rotation(hand_landmarks.landmark)
                    if direction == 'right':    # 오른쪽으로 회전했을 때 방향에 맞춰 회전하기
                        color = BLUE 
                        if counter % 12 == 0:
                            game.rotate(direction)                        
                    elif direction == 'left':    # 왼쪽으로 회전 했을 때 방향에 맞춰 회전하기
                        color = BLUE
                        if counter % 12 == 0: 
                            game.rotate(direction)
                    else:
                        color = RED   
                    STOP = True # 주먹쥐고 있을때 멈춤 True
                
                else:
                    color = GREEN
                    STOP = False # 주먹이 아닐때 멈춤 False
                     # 랜드마크의 좌표를 Pygame 좌표로 변환
        
                for i in range(num_landmarks):
                    hand_x[i] = int(hand_landmarks.landmark[i].x * size[0])
                    hand_y[i] = int(hand_landmarks.landmark[i].y * size[1])

        # OpenCV에서 Pygame 이미지로 변환
        frame_rgb = cv2.flip(frame_rgb, 1)  # 좌우 대칭 (카메라 좌우 반전)
        frame_rgb = cv2.transpose(frame_rgb)  # 전치 (90도 회전)
        frame_surface = pygame.surfarray.make_surface(frame_rgb)
        
        # Pygame 화면 크기에 맞게 프레임 크기 조정
        frame_surface = pygame.transform.scale(frame_surface, size)
        
        # Pygame 화면에 배경으로 그리기
        screen.blit(frame_surface, (0, 0))
            
        # Pygame 화면에 랜드마크 점 그리기
        for i in range(num_landmarks):
            pygame.draw.circle(screen, color, (size[0] - hand_x[i], hand_y[i]), 5)
            
        for i in range(game.height): # 그리드(격자 선) 그리기
            for j in range(game.width):
                pygame.draw.rect(screen, GRAY, [game.x + game.zoom * j, game.y + game.zoom * i, game.zoom, game.zoom], 1)
                if game.field[i][j] > 0:
                    pygame.draw.rect(screen, colors[game.field[i][j]],
                                     [game.x + game.zoom * j + 1, game.y + game.zoom * i + 1, game.zoom - 2, game.zoom - 1])
    
        if game.figure is not None: #피규어(블록) 그리기
            for i in range(4):
                for j in range(4):
                    p = i * 4 + j
                    if p in game.figure.image():
                        pygame.draw.rect(screen, colors[game.figure.color],
                                         [game.x + game.zoom * (j + game.figure.x) + 1,
                                          game.y + game.zoom * (i + game.figure.y) + 1,
                                          game.zoom - 2, game.zoom - 2])
    
        font = pygame.font.SysFont('Calibri', 25, True, False)
        font1 = pygame.font.SysFont('Calibri', 65, True, False)
        text = font.render("Score: " + str(game.score), True, BLACK)
        text_game_over = font1.render("Game Over", True, (255, 125, 0))
        text_game_over1 = font1.render("Press ESC", True, (255, 215, 0))
        text_game_work1 = font.render("Grab to move", True, (255, 215, 0))
        text_game_work2 = font.render("or rotate the block", True, (255, 215, 0))

        screen.blit(text, [20, 0])
        screen.blit(text_game_work1, [size[0] - 300, 0])
        screen.blit(text_game_work2, [size[0] - 300, 20])
        if game.state == "gameover":
            screen.blit(text_game_over, [20, 200])
            screen.blit(text_game_over1, [25, 265])
        
        pygame.display.flip()
        clock.tick(fps)
        
        # 'q' 키를 누르면 종료 (옵션)
        keys = pygame.key.get_pressed()
        if keys[pygame.K_q]:
            break
            
# 웹캠과 Pygame 종료
webcam.release()
pygame.quit()

C:\Users\user\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


SystemExit: 